# Kubernetes Autoscaling Demo with Kind
Project by Martin, Jan and Fynn

This notebook contains the step-by-step commands to set up and demonstrate
Kubernetes autoscaling using Kind (Kubernetes in Docker).

In [ ]:
import subprocess
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Step 1: Set up Kind cluster
!kind create cluster --config three_node_config.yaml

In [ ]:
# Step 2: Build and load Docker image
!docker build -t webapp:latest .
!kind load docker-image webapp:latest

In [ ]:
# Step 3: Apply Kubernetes configurations
!kubectl apply -f metrics-server.yaml
!kubectl apply -f webapp-deployment.yaml
!kubectl apply -f webapp-hpa.yaml
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.7.0/aio/deploy/recommended.yaml

In [ ]:
# Step 4: Wait for pods to be ready
!kubectl wait --for=condition=Ready pods --all --timeout=300s

In [ ]:
# Step 5: Check the status of pods
!kubectl get pods

In [ ]:
# Step 6: Check the status of services
!kubectl get services

### Setup for dashboard

In [ ]:
!kubectl create serviceaccount dashboard-admin -n kubernetes-dashboard
!kubectl create clusterrolebinding dashboard-admin-binding \
  --clusterrole=cluster-admin \
  --serviceaccount=kubernetes-dashboard:dashboard-admin

# safe it in a variable
token = !kubectl -n kubernetes-dashboard create token dashboard-admin
print(f"token: {token}")

# run in a separate terminal
subprocess.run(["kubectl", "proxy"])
print("Dashboard is running at http://localhost:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/")


In [ ]:
# Step 7: Set up port forwarding (run this in a separate terminal)
# !kubectl port-forward service/webapp-service 8080:80

In [ ]:
# Step 8: Monitor HPA (run this in a separate terminal)
# !watch -n 1 kubectl get hpa webapp-hpa


In [ ]:

# Step 9: Generate load (run this in a separate terminal)
# !ab -n 1000 -c 10 http://localhost:8080/
# => full load with max 10 pods

# !ab -n 1000 -c 3 http://localhost:8080/
# => 7 pods 30% cpu


In [ ]:

# Step 10: Check the status of pods again to see scaling
!kubectl get pods

# Step 11: Check HPA status
!kubectl get hpa

# Step 12: Clean up 
#!kind delete cluster